In [1]:
import pysam
import pandas as pd

### Zadatak 1. Koliko egzona poseduje gen FKBP1A?

 - ucitati chr20.sorted.gtf.gz fajl koristeci f-ju *TabixFile* iz biblioteke pysam

In [ ]:
gtf_file = pysam.TabixFile('/sbgenomics/project-files/chr20.sorted.gtf.gz')

 - koristeci f-ju *fetch* dohvatiti sve linije iz GTF fajla koje sadrze ime gena *FKBP1A*

In [3]:
# collect all records of FKBP1A gene?
FKBP1A = []
for gtf in gtf_file.fetch("20", parser=pysam.asGTF()):
    if gtf.gene_name == 'FKBP1A':
        FKBP1A.append(gtf)

 - dohvatiti samo one linije iz GTF fajla koje sadrze informacije o egzonima gena *FKBP1A*

In [4]:
# collect all exone IDs of FKBP1A exones?
exon_ids = []
for gtf in FKBP1A:
    if gtf.feature == 'exon':
        exon_ids.append(gtf.exon_id)

 - prebrojati jedinstvene egzone gena *FKBP1A*

In [5]:
# count the exones with unique IDs
len(set(exon_ids))

30

### Zadatak 2. Odrediti proteinsku sekvencu transkripta ENST00000618612 gena FKBP1A?

 - koristeci f-ju *fetch* dohvatiti linije iz GTF fajla koje sadrze ime transkripta ENST00000618612. Dodatno, izabrati samo one linije koje sadrze informaciju o start kodonu (*start_codon*), stop kodonu (*stop_codon*) i kodirajucim sekvencama (*CDS*).

In [6]:
df_list = []
for gtf in gtf_file.fetch("20", parser=pysam.asGTF()):
    if (gtf.feature == 'start_codon' or gtf.feature == 'CDS' or gtf.feature == 'stop_codon') and gtf.transcript_id == 'ENST00000618612':
        df_list.append({'transcript_id': gtf.transcript_id, 'feature': gtf.feature, 
                        'start': gtf.start, 'end': gtf.end})

In [7]:
df_list

[{'transcript_id': 'ENST00000618612',
  'feature': 'stop_codon',
  'start': 1370028,
  'end': 1370031},
 {'transcript_id': 'ENST00000618612',
  'feature': 'CDS',
  'start': 1370031,
  'end': 1370072},
 {'transcript_id': 'ENST00000618612',
  'feature': 'CDS',
  'start': 1372075,
  'end': 1372240},
 {'transcript_id': 'ENST00000618612',
  'feature': 'CDS',
  'start': 1392833,
  'end': 1392881},
 {'transcript_id': 'ENST00000618612',
  'feature': 'CDS',
  'start': 1392961,
  'end': 1392998},
 {'transcript_id': 'ENST00000618612',
  'feature': 'start_codon',
  'start': 1392995,
  'end': 1392998}]

 - upisati informacije o transkriptu ENST00000618612 u DataFrame

In [8]:
df = pd.DataFrame(df_list, columns= ['transcript_id', 'feature', 'start', 'end'])

In [9]:
df_ENST00000618612 = df[df['transcript_id'] == 'ENST00000618612'].sort_values(by=['transcript_id', 
                                                                                  'start'], ascending=False)

In [10]:
df_ENST00000618612

,transcript_id,feature,start,end
5,ENST00000618612,start_codon,1392995,1392998
4,ENST00000618612,CDS,1392961,1392998
3,ENST00000618612,CDS,1392833,1392881
2,ENST00000618612,CDS,1372075,1372240
1,ENST00000618612,CDS,1370031,1370072
0,ENST00000618612,stop_codon,1370028,1370031


 - izvuci startne i krajnje pozicije za sve **feature** u tabeli

In [11]:
start = []
end = []
for index, row in df_ENST00000618612.iterrows():
    if row['feature'] == 'CDS':
        start.append(row['start'])
        end.append(row['end'])

In [12]:
start, end

([1392961, 1392833, 1372075, 1370031], [1392998, 1392881, 1372240, 1370072])

 - ucitati chr20.fasta fajl koristeci f-ju *FastaFile* iz biblioteke pysam

In [13]:
# reading human chr20 fasta
chr20_fasta = pysam.FastaFile('/sbgenomics/project-files/chr20.fasta')

In [14]:
# function that returns a reverse complement of the given sequence
def revc(dna):
    """Reverse complement a DNA sequence."""
    revc = dna[::-1].translate(str.maketrans('ACGT','UGCA'))
    return(revc)

In [15]:
# function that translates rna to protein sequence
def rna2prot_single(seq):
    protein = ''
    i = 0
    protein1 = ''
    while i <= len(seq)-3:
            protein1 += codon_map[seq[i:i+3]]
            i += 3
    return protein1 

In [16]:
# codon map for translation
codon_map = {"UUU":"F", "UUC":"F", "UUA":"L", "UUG":"L",
    "UCU":"S", "UCC":"S", "UCA":"S", "UCG":"S",
    "UAU":"Y", "UAC":"Y", "UAA":"STOP", "UAG":"STOP",
    "UGU":"C", "UGC":"C", "UGA":"STOP", "UGG":"W",
    "CUU":"L", "CUC":"L", "CUA":"L", "CUG":"L",
    "CCU":"P", "CCC":"P", "CCA":"P", "CCG":"P",
    "CAU":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
    "CGU":"R", "CGC":"R", "CGA":"R", "CGG":"R",
    "AUU":"I", "AUC":"I", "AUA":"I", "AUG":"M",
    "ACU":"T", "ACC":"T", "ACA":"T", "ACG":"T",
    "AAU":"N", "AAC":"N", "AAA":"K", "AAG":"K",
    "AGU":"S", "AGC":"S", "AGA":"R", "AGG":"R",
    "GUU":"V", "GUC":"V", "GUA":"V", "GUG":"V",
    "GCU":"A", "GCC":"A", "GCA":"A", "GCG":"A",
    "GAU":"D", "GAC":"D", "GAA":"E", "GAG":"E",
    "GGU":"G", "GGC":"G", "GGA":"G", "GGG":"G",}

 - koriscenjem rna2prot_single i revc f-ja pretvoriti sekvencu DNK u sekvencu aminokiselina

In [17]:
peptides = []
for i in range(len(start)):
    peptides.append(rna2prot_single(revc(chr20_fasta.fetch('20', start[i], end[i]))))

In [18]:
''.join(peptides)

'MGVQVETISPGDGAPSPSAARPAWCTTPMSVGQRAKLTISPDYAYGATGHPGIIPPHATLVFDVELLKLESTOPQEWPPPLAPCSWICHGGIWCLQTCA'